# Prediciting Building Damage in Kavrepalanchok

In [ ]:
# Import Librairies
import warnings
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from category_encoders import OneHotEncoder
from category_encoders import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree

## Prepare data

### Connect

In [ ]:
%load_ext sql
%sql sqlite:////home/jovyan/<fileName e.g nepal>.sqlite

# sample output
'Connected: @/home/jovyan/nepal.sqlite'

In [ ]:
# Get districts that are represented in the id_map table
%%sql
SELECT distinct district_id
FROM id_map
LIMIT 5

# sample output 
Running query in 'sqlite:////home/jovyan/nepal.sqlite'
district_id
1
2
3
4

In [ ]:
# Get number of observations in table id_map for district 3
%%sql
SELECT count(*)
FROM id_map
WHERE district_id = 3

# sample output
Running query in 'sqlite:////home/jovyan/nepal.sqlite'
count(*)
82684

In [ ]:
%%sql
# joining tables at columns building_id
SELECT distinct(i.building_id) AS b_id,   # building_id column of table i aliased as b_id
     s.*,     # selects all columns of table s
     d.damage_grade   # select damage_grade column of table d
FROM id_map AS i
JOIN building_structure AS s ON i.building_id = s.building_id
JOIN building_damage AS d ON i.building_id = d.building_id
WHERE district_id = 3
LIMIT 5

# sample output: refer to Figure 3.1 

### Import

In [ ]:
def wrangle(db_path):
    # Connect to database using connect method
    conn = sqlite3.connect(db_path)

    # Construct query
    query = """
        SELECT distinct(i.building_id) AS b_id,
           s.*,
           d.damage_grade
        FROM id_map AS i
        JOIN building_structure AS s ON i.building_id = s.building_id
        JOIN building_damage AS d ON i.building_id = d.building_id
        WHERE district_id = 3
    """

    # Read query results into DataFrame
    df = pd.read_sql(query, conn, index_col="b_id")

    # Identify leaky columns
    drop_cols = [col for col in df.columns if "post_eq" in col]
    
    # Create binary target
    df["damage_grade"] = df["damage_grade"].str[-1].astype(int)
    df["severe_damage"] = (df["damage_grade"] > 3).astype(int)    # encode as 0's and 1's
    
    # Drop old target
    drop_cols.append("damage_grade")
    
    # Drop multicolinearity column
    drop_cols.append("count_floors_pre_eq")
    
    # Drop high categorical features
    drop_cols.append("building_id")
    
    # Drop columns
    df.drop(columns=drop_cols, inplace=True)
    
    
    return df

In [ ]:
# Using wrangle func
df = wrangle("/home/jovyan/nepal.sqlite")
df.head()

# sample output: refer to figure 3.2

### Explore

In [ ]:
# Check to see if dataset is balanced 
# Plot value counts of `"severe_damage"`
df["severe_damage"].value_counts(normalize=True).plot(
    kind="bar", xlabel="class", ylabel="rel freq", title="class bal")

# sample output: refer to figure 3.3
# Insights

In [ ]:
# Check to see if there a relationship between the footprint size of a building and the damage it sustained in the earthquake
sns.boxplot(x="severe_damage", y="plinth_area_sq_ft", data=df)
# Label axes
plt.xlabel("Severe Damage")
plt.ylabel("Plinth Area [sq. ft.]")
plt.title("Kavrepalanchok, Plinth Area vs Building Damage");

# sample output: refer to figure 3.4
# Insights

In [ ]:
# Check to see if buildings with certain roof types more likely to suffer severe damage
roof_pivot = pd.pivot_table(
    df, index="roof_type", values="severe_damage", aggfunc=np.mean    # roof_type: column in table
).sort_values(by="severe_damage")
roof_pivot

# sample output: refer to figure 3.5
# Insights

### Split

In [ ]:
# Vertical Split
target = "severe_damage"
X = df.drop(columns=target) # feature matrix: all columns apart from severe_damage
y = df[target]
print("X shape:", X.shape)
print("y shape:", y.shape)

# sample output:
X shape: (76533, 11)
y shape: (76533,)

In [ ]:
#Horizontal Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

# sample output:
X_train shape: (61226, 11)
y_train shape: (61226,)
X_val shape: (15307, 11)
y_val shape: (15307,)

## Build Model

### Baseline

In [ ]:
acc_baseline = y_train.value_counts(normalize=True).max()
print("Baseline Accuracy:", round(acc_baseline, 2))

# sample output:
Baseline Accuracy: 0.55

### Iterate

In [ ]:
model_lr = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    LogisticRegression(max_iter=<1000-3000) # max_iter: varies: suppresses the 'ConvergenceWarning'
)
# Fit model to training data
model_lr.fit(X_train, y_train)

In [ ]:
# model_lr training and validation accuracy scores 
lr_train_acc = accuracy_score(y_train, model_lr.predict(X_train))
lr_val_acc = model_lr.score(X_val, y_val)

print("Logistic Regression, Training Accuracy Score:", lr_train_acc)
print("Logistic Regression, Validation Accuracy Score:", lr_val_acc)

sample output:
Logistic Regression, Training Accuracy Score: 0.6515720702327481
Logistic Regression, Validation Accuracy Score: 0.649436550710436

In [ ]:
# Testing to seeing if a decision tree model will perform better than logistic regression
depth_hyperparams = range(1, 16)
training_acc = []
validation_acc = []
# Trains and eval the model model_dt at all depths from 1 to 15 to find the best hyperparameter value for max_depth
for d in depth_hyperparams:
    model_dt = make_pipeline(
        OrdinalEncoder(), 
        DecisionTreeClassifier(max_depth= d, random_state=42)
    )
    model_dt.fit(X_train, y_train) # Fit model to training data
    
    # Calculate training accuracy score and append to `training_acc`
    training_acc.append(model_dt.score(X_train, y_train))
    # Calculate validation accuracy score and append to `training_acc`
    validation_acc.append(model_dt.score(X_val, y_val))
    
print("Training Accuracy Scores:", training_acc[:6])
print("Validation Accuracy Scores:", validation_acc[:6])

# sample output:
Training Accuracy Scores: [0.6303041191650606, 0.6303041191650606, 0.642292490118577, 0.653529546271192, 0.6543951915852743, 0.6576617776761506]
Validation Accuracy Scores: [0.6350035931273273, 0.6350035931273273, 0.6453909975828053, 0.6527732410008493, 0.6529039001763899, 0.6584569151368654]    

In [ ]:
# Validation curve
plt.plot(depth_hyperparams, training_acc, label="Training")
plt.plot(depth_hyperparams, validation_acc, label="validation")
plt.xlabel("Max Depth")
plt.ylabel("Accuracy Score")
plt.title("Validation Curve, Decision Tree Model")
plt.legend();

# sample output: refer to Figure 3.6
# Insights:

In [ ]:
# build & fit again
final_model_dt = make_pipeline(
    OrdinalEncoder(), 
    DecisionTreeClassifier(max_depth=10, random_state=42)
)
# Fit model to training data
final_model_dt.fit(X, y)    #final_model_dt.fit(X_train, y_train)

### Evaluate

In [ ]:
# test type 1
X_test = pd.read_csv("data/kavrepalanchok-test-features.csv", index_col="b_id")
y_test_pred = pd.Series(final_model_dt.predict(X_test))
y_test_pred[:5]

# sample output
0    1
1    1
2    0
3    1
4    0
dtype: int64


# test type 2
test_acc = model.score(X_test, y_test)
print("Test Accuracy:", round(test_acc, 2))
  
# sample output
Test Accuracy: 0.72
    

# test type 3
acc_train = accuracy_score(y_train, model_lr.predict(X_train))
acc_test = model_lr.score(X_test, y_test)

print("LR Training Accuracy:", acc_train)
print("LR Validation Accuracy:", acc_test)

# sample output
LR Training Accuracy: 0.717985042664646
LR Validation Accuracy: 0.7218817948211109

## Communicate

In [ ]:
# DECISION TREE
features = X_train.columns
importances = <yourModel eg final_model_dt>.named_steps["decisiontreeclassifier"].feature_importances_
feat_imp = pd.Series(importances, index=features).sort_values()
feat_imp.head()

# sample output
plan_configuration        0.004032
position                  0.007129
land_surface_condition    0.008241
ground_floor_type         0.009741
foundation_type           0.010620
dtype: float64


# LOGISTIC REG
features = model_lr.named_steps["onehotencoder"].get_feature_names()
importances = model_lr.named_steps["logisticregression"].coef_[0]
feat_imp = pd.Series(np.exp(importances), index=features).sort_values()
feat_imp.head()

# sample output
superstructure_Brick, cement mortar    0.345719
foundation_type_RC                     0.364478
roof_type_RCC/RB/RBC                   0.415979
ground_floor_type_RC                   0.527756
caste_household_Kumal                  0.543642
dtype: float64

In [ ]:
# horizontal bar chart of feature importances
feat_imp.plot(kind="barh")
plt.xlabel("importance")
plt.ylabel("Label")
plt.title("Feature Importance");

# sample output: refer to Figure 3.7
# insights: